<font size="6"><center>OFF GRID HOUSING ENERGY</center></font>
<font size="3"><center>Lauren Heintz, Gabriel Pascualy, Logan Greene, Lisa Schleuter, Jomi Kramer, Nick Esposito</center></font>
<font size="3"><center>LGO 2023</center></font>
<font size="3"><center>August 14, 2021</center></font>

In [1]:
%%capture
from offgridoptimizer import OffGridOptimizer
ogo = OffGridOptimizer()
ogo.load_sheets(None)

ModuleNotFoundError: No module named 'tabulate'

<font size="5"><center>I. Introduction</center></font>

As the effects of climate change become increasingly apparent, people are looking towards alternative energy and ways to live off the grid. Living off the grid means that you do not rely on public utilities for utilities like electricity, running water, sewer, natural gas, or heat, but rather you use independent sources. Because it is expensive to develop your own system to live off the grid, people take a hybrid approach to alternative energy, meaning that they utilize small capacity renewable energy products while still utilizing minimal public utilities.    

For many Americans, the high capital costs of alternative energy products are an obstacle to transition to low-carbon electricity. The amortization of the initial investment for off-grid energy must be cheaper than the cost of continuing to pay for grid-energy.  Off-grid energy has become more popular recently due to increasing ease-of-use for non-commercial use, and an increased awareness of energy consumption and longevity of resources.  Over time, the financial benefit of renewable energy products can save a substantial amount of money on utilities while minimizing the impact on the environment. Just a decade ago, it was much cheaper to build a new coal-burning power plant than it was to build a large capacity wind or solar plant. Since then, the price per MWh has declined by 89% for large capacity renewable energy sources which has trickled down into the smaller capacity renewable energy product market [(Roser, 2020)](#ref1). Recently, reductions in the initial cost of renewable energy products have been spurred on by improvements in manufacturing processes and advances in technology which have made living off the grid more accessible to a wider range of income levels. As of 2017, approximately 180,000 families live off the grid which is expected to significantly increase every year [(Infoplease, 2020)](#ref2).  

Our project aims to assist families considering living off the grid for their electricity consumption. Our goal is to develop a tool that will help evaluate how to minimize the overall cost associated with off-grid products based on the energy resources available, regional efficiencies of wind and solar energy, and costs of various energy products. The tool will accept inputs for average electricity demand, regional information associated with production of wind and solar energy, initial and monthly budget constraints, and costs associated with all energy products. Based on these parameters, the user will be able to determine the optimal mix of energy products (including utilizing the grid) to meet demand while minimizing financial costs.   

<font size="5"><center>II. Background</center></font>

<font size="4"><i>A. Demand</i></font>

This optimization problem starts with energy demand data. We chose to use hourly electricity demand data over the course of the year in units of kwh (kilowatt-hours). This data was procured from [(EIA - US Energy Information Administration, 2021)](#ref4) which gives total grid energy consumption for various regions of the United States. Using information on the percent breakdown of residential energy usage from the grid, and census data on the population of the region, we derived an average hourly energy consumption metric per household, per hour, for every hour of a standard 365 day calendar year, using the readily available data for year 2020 (removing February 29th since 2020 is a leap year). This estimation method aligned closely with the total estimated energy consumption per household from [(EIA - US Energy Information Administration, 2021)](#ref5). 

Then, the average hourly electricity demand for each day was utilized in the model to capture the changes in demand during different seasons and for different parts of the country with different weather patterns. The user interface in this notebook allows the user to choose from a few predefined locations in a drop-down list which capture demand data from different regions of the United States. Users can also procure and upload their own demand data in the format required for the model.  Because of the number of variables and the computational limits of the model, the hourly yearly demand is sampled for every other hour, totalling to 4380 (365 * 12) hours per year.

Additionally, we provide a feature which allows the user to choose the times of year during which the home would be used. For a full-time resident, this would be the full year. However, if this off-grid house is a summer vacation home or a winter vacation home, that may change the energy demand needs. As a result, the recommended product suggestions output from our model will change according to this data. 


In [2]:
ogo.plot_demand_month(['January', 'April', 'July'], 'Cambridge,MA')

NameError: name 'ogo' is not defined

A complementary set of data required to supplement this energy demand data is the data set describing the efficiency of different energy generation products based on geographical location. The efficiency of solar and wind can vary greatly based on the time of year, the amount of sunlight a region receives, temperature, and local weather patterns for that area. We acquired data on the efficiency of solar power and wind power from [(Matasci, 2021)](#ref6) which allows an input of any city, and in return provides hourly efficiency data. This data is used in the optimization to understand the optimal mix of energy products to support the user’s home, depending on the location.


In following plot of solar efficiency in Greenville, ME, the change in solar efficiency is apparent across January, April, and July. Peak solar efficiency is lower in the winter month of January, which makes sense based on weather patterns and the angle of the earth, which affect the intensity of the UV rays. The width of the spike is smaller in January than in July or April, indicating a shorter winter day.

In [3]:
ogo.plot_solar_efficiency_month(['January', 'April', 'July'], 'Greenville,ME')

The following plot shows the variabiltiy of wind efficiency in Greenville, ME across January, April, and July. During the winter and spring months the wind efficiency is elevated and storms in January and April enabled wind products to operate at almost 100% efficiency.

In [4]:
ogo.plot_wind_efficiency_month(['January', 'April', 'July'], 'Greenville,ME')

<font size="4"><i>B. Supply</i></font>


In the optimization model, the supply of energy to meet the demand comes from several possible power sources. These can first be divided by on-grid and off-grid power sources. Within off-grid power sources, we have energy generation products and energy storage products. The two types of energy generation products are wind and solar power. The table below defines the mix of products we chose to input into our model, so the output of the model provides an optimal combination of these power sources that meets energy demand and other constraints, while minimizing grid usage and cost to the resident.       
  
Product |Generation/Storage |Energy Type |On-Grid/Off-Grid
-----|-----|-----|----| 
**Grid Power**|Generation|Grid|On-Grid
**Solar S**|Generation|Solar|Off-Grid
**Solar L**|Generation|Solar|Off-Grid  
**Wind Turbine S**|Generation|Wind|Off-Grid
**Wind Turbine L**|Generation|Wind|Off-Grid
**Battery**|Storage|Storage|Off-Grid

<center> Table 1: Available Products </center>

Each of these products is defined with a maximum amount of energy output in kwh (kilowatt-hours). These detailed specifications are defined in a configuration JSON file which can be uploaded by the user for custom products they would like to consider. Each power generation product (solar and wind) has a maximum capacity in kwh, which is multipled by an efficiency metric mentioned in the demand section above. This provides the expected hourly power generation in kwh of each product over every hour of the year. 


The energy generation products and storage products interact directly with each other. In our case, "Storage" products are batteries and can be used to store excess energy generated, or they can be used to meet the energy demand of the customer. Our system is set up so that excess energy generated daily from wind and solar products can be stored indefinitely and used for other days when demand is higher. We operate under the assumptions that you would not pull energy from the grid and store it on your batteries, and that the intention of using battery energy is to minimize the amount of on-grid energy consumed. Of practical note is that net-metering is a popular option for households that produce alternative energy and do not want to invest in batteries. Excess energy is contributed back to the grid, and the customer builds an “energy credit” for future use. Net-metering is different from selling energy to the grid which, depending on the state, could be less profitable. An important note is that in the case of net-metering, the amount of energy 'sold back' to the grid cannot exceed the households demand.  


In [5]:
ys = ['demand', 'capacity', 'storage_level', 'grid_usage']
ogo.plot(ys, 'July', [5, 6, 7], 'Asheville, NC', budget='Low')

Our model enables the user to select whether to allow a grid connection. If the user chooses to allow connection to the grid, the model defaults to net-metering. In the chart above, energy stored increases during periods of high capacity and depletes again when demand exceeds the capacity available from the wind and solar products. In some cases, grid usage may still be required in sustained high demand periods.  

In [6]:
ys = ['demand', 'storage_level', 'capacity', 'energy_sold']
ogo.plot(ys, 'July', [13, 14, 15, 16], 'Asheville, NC', budget='High', sellback=True)

As seen above, in the case where a household can sell back indefinite amounts of energy to the grid, the model is incentivized to expand capacity for all wind and solar products and always sell back any excess not consumed by the demand. This leads to more batteries purchased and more energy generations products purchased to supplement energy production.

A household's ability to satisfy demand requirements is a combination of the energy generation products and the storage products. The output of our tool provides the user with a recommended selection of products which optimizes cost. An example of this output is displayed in Table 2 & 3.    

<br>

Product |Units to Purchase|
-----|-----|
**Solar S**|1|
**Solar L**|1|  
**Wind Turbine S**|1|
**Wind Turbine L**|1|
**Battery**|1|

<center> Table 2: Selected Products </center>

Grid Usage |Kilowatt-Hours|
-----|-----|
**Grid Energy Consumed**|1|
**Energy Sold to Grid**|1|

<center> Table 3: Grid Usage </center>

<font size="4"><i>C. Cost</i></font>

<u>Off Grid Product Cost Structure</u>  

Each product from the product list has a set of cost variables associated with it that are defined below:
- **"Opening Cost"**: The baseline set up cost to install the wiring and necessary equipment to connect an external energy source (solar, wind, batteries) to a house in a way that can be used by the household. This would only be done once for each type of energy source.
- **"Unit Cost"**: The cost per unit of a product, such as a solar panel or a wind turbine, that is incurred once per unit at the time of initial purchase.
- **"Maintenance Cost"**: The estimated annual cost to upkeep and maintain a given product from the product list.

<u>Grid Connection Cost Structure</u> 
- **"Opening Cost"**: The baseline set up cost to install the wiring and necessary equipment to connect the house the nearest grid power source, assuming that it is not yet connected.
- **"Grid Hourly Cost"**: The cost per kilowatt-hour to pull energy directly from the grid and use it in a home.
- **"Grid Hourly Revenue"**: The price per kilowatt-hour the excess energy can be sold back to the grid.

<font size="5"><center>III. MIP Formulation</center></font>

<font size="4"><i>A. Demand</i></font>

Let $h$ := be the hour of the year where total hours in a year = (24*365), $h \in H$ where $H = \{{0, 1, 2, ... 8760}\}$ (in final formulation, only every other hour was sampled if over 6 months due to space constraints for decision variables).

Let $d_{h}$:= be the customer electricity demand in KwH during hour $h$ of the year where $d_{h} \geq 0$

<font size="4"><i>B. Power Source Parameters</i></font>

<font size="3"><i>1) Product Definition Parameters</i></font>

Let $j$ := be a product $j$ $\in$ $J$ where $J = \{{"SmallSolar", "LargeSolar", "SmallWind", "LargeWind", "Battery", "Grid"}\}$

Let $E$ := be a set containing indices of products which generate electricity except for "grid"

Let $S$ := be a set containing indices of products which store electricity 

Let $et$ := be an energy type where $et \in T = \{{"solar", "wind", "storage"}\}$

Let $ET_{et}$ := be a set containing all products of energy type $et$

<font size="3"><i>2) Cost Paramaters</i></font>

Let $oc_{j}$ := be the opening cost in US dollars of installing required infrastructure for product $j$ which belongs to an energy type $et$

Let $uc_j$ := be the unit cost in US dollars of purchasing an additional unit of a specific product $j$ where $uc_j \geq 0$

Let $mc_j$ := be the cost in US dollars of maintaining one unit of product $j$ where $mc_j \geq 0$

Let $Gc_h$ := be the cost of on-grid power in units of dollers per kilowatt hour for hour $h$  

Let $Gr$ := be the revenue rate in dollars per KwH that excess energy can be sold back to the grid. This number represents positive revenue.

<font size="3"><i>3) Energy Conversion and Storage Parameters</i></font>

Let $\rho_{h,j}$ := be the efficiency of product $j$ during hour $h$ of the year where $\rho_{h,j}$ is continuous from 0 to 1

Let $c_{j}$ := be the capacity of product $j$ in KwH if it generates or stores electricity


<font size="4"><i>C. Decision Variables</i></font>

<font size="3"><i>1) Product Decision Variables</i></font>

Let $x_j$ := be a binary variable representing whether or not the opening cost of $j$'s energy type is paid for where $x_j \in$ {0,1}

Let $y_{j}$ := be an integer variable representing the number of units of product $j$ where $y_j \geq$ 0

<font size="3"><i>2) Grid Power Decision Variables</i></font>

Let $g_{h}$ := be the continuous variable that represents the amount of power needed from the grid in kilowatt-hours during hour $h$ of the year.

Let $ge_{h}$ := be the continuous variable that represents the amount of excess energy sold back to the grid in kilowatt-hours during hour $h$ of the year.

<font size="3"><i>3) Storage Decision Variables</i></font>

Let $es_{h,j}$ := be the continuous variable that represents the amount of energy stored by product $j$ (note: not all products can store energy)

Let $sc_{h,j}$ := be the continuous variable that represents the amount of stored consumed from product $j$ (note: not all products can store energy)

<font size="4"><i>D. Calculated Variables</i></font>

<font size="3"><i>1) Calculated Storage Variables</i></font>

Let $ES_{h}$ := be the total energy stored this year through hour $h$ calculated by summing cumulative the energy stored each hour for all products $\sum_{j \in J} es_{h,j}$

Let $SC_{h}$ := be the total storage consumed this year through hour $h$ calculated by summing the cumulative energy consumed each hour for all products $\sum_{j \in J} sc_{h,j}$

Let $TT$ := be the total energy storage of the system calculated by summing the capacity of all products $\sum_{j \in J} c_{j}y_{j}$



<font size="4"><i>E. Objective Function</i></font>

We want to minimize the cost; therefore, the objective function is the sum of various costs:

    a) opening cost (oc) the cost of initially setting up a new service

    b) unit cost (uc)
    
    c) maintenance cost (mc)

    d) cost of grid/hr (Gc) * grid hours consumed (g)

after subtracting the revenue from excess energy sold back to the grid:
    
    e) grid sell back rate (Gr) * energy sold to grid (ge)

$$\min \sum_{j \in J} oc_{j}x_j + \sum_{j \in J} uc_jy_j + \sum_{j \in J} mc_{j}y_{j} + \sum_{h \in H} {(Gc_h)(g_{h}) - (Gr)(ge_h)}$$

<font size="4"><i>F. Constraints</i></font>

<font size="3"><i>1) Demand Constraints</i></font>

1) Meet customer demand for electricity per hour per month by summing the energy generation products ($E$) output, the stored energy ($S$) available, and grid power ($g$) available

$$ \forall h \in H:  \sum_{j\in E} \rho_{h,j} y_{j} + \sum_{j\in S}sc_{h,j} + g_{h} \geq d_{h}$$

<font size="3"><i>2) Budget Constraints</i></font>

2) You have an initial budget of the amount of capital you have on hand to shell out for initial set up.
$$  \sum_{j \in J} oc_{j}x_j + \sum_{j \in J} uc_jy_j \leq $200,000 $$

3) You also have a monthly budget.
$$ \sum_{j \in J} mc_{j}y_{j} \leq $500 $$

<font size="3"><i>3) Set up and Installation Constraints</i></font>

4) Ensure at least 1 $x_j$ per energy type is 1 when any $y_j$ of the energy type is $\geq 1$

$$ \forall j \in ET_{solar}: \sum_{j' \in ET_{solar}} x_{j'} * M >= y_{j}$$

$$ \forall j \in ET_{wind}: \sum_{j' \in ET_{wind}} x_{j'} * M >= y_{j}$$

$$ \forall j \in ET_{storage}: \sum_{j' \in ET_{storage}} x_{j'} * M >= y_{j}$$

5) Ensure that at most 1 $x_j$ per energy type is equal to 1

$$ \forall et \in ET: \sum_{j\in ET_{et}}x_j \leq 1$$

<font size="3"><i>4) Storage Constraints</i></font>

6) Total storage consumed at hour $h$ of the year cannot exceed electricity stored

$$ \forall h \in H\space: SC_{h} \leq ES_{h} $$ 

7) Storage available at hour $h$ of the year cannot exceed total storage capacity

$$ \forall h \in H\space: ES_{h} - SC_{h}  \leq TT $$

8) Electricity stored at hour $h$ of the year cannot exceed the sum of electricity produced, storage consumed, and grid usage minus demand at hour $h$ of the year and energy sold back to the grid at hour $h$. 

$$ \forall h \in H:\space \sum_{j\in S} es_{h,j} \leq g_{h} - d_{h} - ge_h + \sum_{j\in E} \rho_{h,j} y_{j} + \sum_{j\in S} sc_{h,j} $$

<font size="3"><i>5) Grid Constraints</i></font>

9) No energy can be sold back to the grid unless the grid is connected/installed to the house.

$$ \forall h \in H:\space ge_h \leq x_{j=grid}*M $$

$$ x_{j=grid} * M >= \sum_{h \in H} g_h$$

<font size="5"><center>IV. Tool Interface</center></font>

Below is the tool our team developed for individuals who need to identify the optimal selection of products for their location and budget constraints.

The tool is loaded with default configurations which include location, budget, and product information. The user can either fill out all information manually or load one of these default configurations and edit fields to match their  scenario. The user can select which months to run the simulation across using the checkboxes. Please note that if the user selects more than 6 months for analysis, the tool will only analyze every other hour of the year. This tool requires an enormous amount of system memory to complete its analysis so to ensure that the optimization completes on a typical laptop this limitation is imposed.

After clicking the optimize button and waiting for the optimization to complete, the selected products table and the associated costs table will be populated by the tool with its findings.

In [7]:
%%capture
ogo.optimize(None)

In [8]:
ogo.interface

Box(children=(Box(children=(HTML(value='<h2>Off-Grid Optimizer</h2>', layout=Layout(height='auto')), Box(child…

In [9]:
ogo.plot_results(ogo.project.results_df())

<font size="5"><center>V. Results</center></font>

<font size="3"><i>1) Budget Constraint Analysis</i></font>

Our team explored the effect of different capital budgets on the optimal product mix for minimizing grid-usage. Three budget options were considered: low-budget, mid-range, and high-end (Table 4). For all of these options, the objective was both to minimize the amount of grid usage required while also minimizing cost. In the specific case of our pilot site (Asheville, NC), the high-end option was the only one that allowed full off-grid living. This demonstrates the increasing cost of independence to live completely off the grid.

<br>

| | Low-End (\\$20k initial, \\$250 yearly) | Mid-Range (\\$40k, \\$250) | High-Budget (\\$50k, \\$500) |
-----|-----|-----|-----|
**Large Solar Panel**| 0 | 0 | 1
**Small Wind Turbine**| 2 | 1 | 1
**Large Wind Turbine**| 0 | 1 | 0
**Batteries**| 0 | 1 | 2
**Grid Usage**| 1000 kWh | 100 kWh | 0
**Grid Sold**| 1000 kWh | 100 kWh | 0
**Total Cost**| \\$20k | \\$40k | \\$50k
**Breakeven**| 11 years | 35 years | 47 years

<center> Table 4: Budgets Constraint Analysis</center>

As a part of each budget constraint analysis, we calculated the anticipated cost of standard on-grid energy given the same annual demand and compared it to partially/completely off-grid option. The break-even point is the time at which the energy savings offset the initial capital investment. For the Asheville site, this was found to be 47 years. The initial capital budget and pay-off periods are the biggest factors that will determine a household’s available product mix. The breakeven points for the Asheville area were fairly high, due to inconsistent wind efficiency and high energy demand in the summer. However, note that most of the population will use alternative energy sources in combination with net-metering, which drastically reduces the pay-off period. Furthermore, this break-even analysis does not include rebates or other tax incentives for clean energy investments.

<font size="3"><i>2) Guranteed Buy Back Rate Analysis</i></font>

<br>

As an extension of the budget constraint analysis, we ran the scenario where the household could sell back energy to the grid at a guaranteed buy back rate (as opposed to net metering). The budget used was the high-end budget from Table 4, as that was the only scenario in which the Asheville off-grid home achieved full grid independence.

| | Guaranteed Buy Back Rate (\\$50k initial, \\$500 yearly) |
-----|-----|
**Small Solar Panel**| 1 |
**Large Wind Turbine**| 1 |
**Batteries**| 1 |
**Grid Usage**| 17 kWh |
**Grid Sold**| 42 mWh |
**Total Cost**| \\$40k |
**Breakeven**| 9 years |

<center> Table 5: Guaranteed Buy Back Rate Analysis</center>

This substantially reduces the break-even point for the capital investment, since more energy can be sold back to the grid. Additionally, the upfront capital investment is lower than the previous model since fewer batteries are needed to store energy. This is because any quantity of energy can immediately be sold back to the grid and doesn't need to be stored for later use. Since this version of the model is seeking to minimize expenses and sell as much energy back as possible, it defaults to buying more energy products than are necessary to meet your demand in order to increase revenue from energy sold back. Some considerations to keep in mind here are whether or not you are ok turning your land in to a small solar or small wind farm just for the purposes of generating revenue.  

<font size="3"><i>3) Seasonal Analysis</i></font>

We also analyzed the effects of seasonal home usage compared to full-year home usage. For the case of our pilot site, this showed both the relative costs of optimizing for various seasons, as well as the most economical mix, dependent on season (Table 6).  

<br>

|(\\$60k Capital, \\$500 Yearly) | Winter | Spring | Summer | Fall |
-----|-----|-----|-----|-----|
**Small Solar Panel**| 0 | 0 | 2 | 0
**Small Wind Turbine**| 3 | 4 | 1 | 2
**Large Wind Turbine**| 1 | 0 | 0 | 1
**Batteries**| 2 | 1 | 2 | 2
**Total Cost**| \\$39k | \\$28k | \\$55k | \\$38k

<center> Table 6: Seasonal Analysis</center>

Unsurprisingly, the summer months are better for solar products, whereas the storms in the winter and spring months are more optimal for wind products. Summer is most expensive, probably due to increased demand for air conditioning. In this instance, spring is the cheapest time of year. The fact that we need fewer batteries in the spring indicates that the wind power is more reliable, and less variable this time of year. 

<font size="3"><i>4) Regional Analysis</i></font>

Finally, we collected data for various regions in the U.S., and highlighted the impact of the three distinct locations in Table 7 below. The differences in both demand and supply make this a complex problem.  This analysis does not consider any state-specific tax breaks, which could greatly influence the optimal product mix (10). Out of the three sites highlighted below, Asheville requires a variety of products and a high quantity of batteries to go off-grid, due to inconsistency in alternative energy efficiencies, as well as high energy demand in the summer. Sedona is a very cost-effective region to go off-grid, as energy sources are very consistent in this area. Yakima is a very windy region and is the site of many commercial wind farms. However, the high variability in wind efficiency requires a high number of batteries. Cost inputs can easily be adjusted in our interface to account for state tax breaks, and this product mix can be further customized. 

<br>

|(\\$60k Capital, \\$500 Yearly) | Asheville, NC | Sedona, AZ | Yakima, WA |
-----|-----|-----|-----|
**Large Solar Panel**| 1 | 0 | 0 |
**Small Wind Turbine**| 1 | 3 | 3 |
**Large Wind Turbine**| 0 | 0 | 1 |
**Batteries**| 2 | 1 | 2 |
**Total Cost**| \\$50k | \\$26k | \\$40k |

<center> Table 6: Regional Analysis</center>


<font size="5"><center>VI. Conclusion</center></font>

This tool can be used and scaled for several different use-cases.  Our focus for this tool was primarily for individual household users. Though our tool does not incorporate any policy-related factors such as tax incentives, this could easily be added. The sensitivities for different regions and demographics that are shown through this tool could be used by policy-makers to determine incentive thresholds to help their area reach certain alternative-energy goals. 

In all, we are encouraged by the results we found from our preliminary model of off-grid energy feasibility. While the fully off-grid solutions require more upfront capital investment, there are still great savings to be had with hybrid solution. Additionally, none of our cost analysis included tax incentives and other rebates related to clean, renewable energy. These are dependent by state, but likely to become more generous over time. Additionally, the cost of renewable energy products has gone down, and will likely continue to become more and more economical - further shortening the break-even point. There is no doubt that there is a cost to independence from the grid, as well as a cost for more sustainable energy. Beyond the purely economical approach, many are driven to diversify their energy sources for the sake of building a more sustainable future for our world. This tool can find several locally optimal solutions, depending on the real objectives of the user and how they manage the inputs (i.e. constraints) to the model.

One observation about this model is that it skews towards wind products far more than solar products. This is the case even in places like Sedona, where conventionally we would lean towards a solar investment. This is likely a byproduct of the input cost data for wind and solar products - solar items may have been overpriced in our model. The user can manually update this cost data in the configuration JSON.

Another point worth noting is that this model does not consider lot size as a factor in the optimization. In reality, that may be in important factor dictating how many solar panels or wind turbines you are willing to purchase. The large wind turbine fan blades are roughly 8 feet in length. This may be an attractive nuisance that is undesired by a household, nearby neighbors, or the surrounding community. In the future, this would be a valuable constraint to add in to the model.

Since several of our team-members plan to build partial or fully off-grid homes in the future, and we also incorporated feedback from people who currently live in partial or fully off-grid homes. This “customer research” helped us incorporate realistic factors and define our decision and static variables. Many classmates, friends, and family members have already reached out to request the use of our tool for evaluating alternative energy for their specific situation. We hope this can provide valuable insights even in its current state!


<font size="5"><left>References</left></font>

<a name="ref1">1.</a> Roser, M. (2020, December 1). Why did renewables become so cheap so fast? And what can we do to use this global opportunity for green growth? Our World in Data. https://ourworldindata.org/cheap-renewables-growth

<a name="ref2">2.</a> Infoplease. (2020, August 5). Living Off the Grid. https://www.infoplease.com/math-science/earth-environment/living-off-the-grid 

<a name="ref3">3.</a> Households By State 2021. (2021). World Population Review. https://worldpopulationreview.com/state-rankings/households-by-state 

<a name="ref4">4.</a> Real-time Operating Grid - U.S. Energy Information Administration (EIA). (2021). Hourly Electric Grid Monitor. https://www.eia.gov/electricity/gridmonitor/dashboard/electric_overview/regional/REG-CAR 

<a name="ref5">5.</a> Electricity use in homes - U.S. Energy Information Administration (EIA). (2021). Use of Energy Explained. https://www.eia.gov/energyexplained/use-of-energy/electricity-use-in-homes.php 

<a name="ref6">6.</a> Matasci, S. (2021, July 29). The cost of solar panels in 2021: what price for solar can you expect? Solar News. https://news.energysage.com/how-much-does-the-average-solar-panel-installation-cost-in-the-u-s/ 

<a name="ref7">7.</a> Literature | Trojan Battery Company. (2021). Trojan Battery Data Sheets. https://www.trojanbattery.com/literature/#datasheets 

<a name="ref8">8.</a> Amazon.com : Automaxx Windmill 1500W 24V 60A Wind Turbine Generator kit. Automatic and Manual Braking System DIY Installation, MPPT Controller with Bluetooth Function & Spare Blade Set : Patio, Lawn & Garden. (2021). Automaxx Windmill 1500W 24V 60A Wind Turbine. https://www.amazon.com/dp/B01ASNP062/?tag=19gh-20 

<a name="ref9">9.</a> Pika Home Wind Turbine /T701. (2017, January). https://www.pika-energy.com/files/datasheets/generation_1/pika_t701_wind_turbine.pdf 

<a name="ref10">10.</a> NC Clean Energy Technology Center. Database of State Incentives for Renewables & Efficiency. (2021) https://programs.dsireusa.org/system/program/nc  

<a name="ref11">11.</a> Hourly Renewable Power Output Simulator. https://www.renewables.ninja/